In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
#get all the data into one big frame
file = "./data/games.csv"
wholeFrame = pd.read_csv(file)

In [3]:
#build the frames we want
moveFrame = wholeFrame.drop(['promote'], axis=1)
promoteFrame = wholeFrame.drop(['from', 'to'], axis=1)

In [4]:
#remove duplicates
moveFrame = moveFrame.drop_duplicates()
promoteFrame = promoteFrame.drop_duplicates()

#remove rows where theres no promotion
promoteFrame = promoteFrame[promoteFrame.promote != 0]

In [5]:
#update indices because we've removed some stuff
moveFrame = moveFrame.reset_index()
promoteFrame = promoteFrame.reset_index()

In [6]:
#shuffle frames
moveFrame = moveFrame.sample(frac = 1)
promoteFrame = promoteFrame.sample(frac = 1)

In [7]:
#function for spliting the board into individual chars
def parseBoard(board):
    return np.asarray([char for char in board]).astype("float32")

In [8]:
#function for converting 'to' and 'from' to formats the nn can use
def parseMove(move):
    result = np.zeros(64,dtype = np.float32)
    result[move] = 1
    return result
    

In [9]:
#function for converting 'promote' to formats the nn can use
def parsePromote(promoteTo):
    result = np.zeros(3,dtype = np.float32)
    result[promoteTo-1] = 1
    return result

In [10]:
#split boards into chars
moveFrame['board'] = moveFrame['board'].apply(parseBoard)
promoteFrame['board'] = promoteFrame['board'].apply(parseBoard)

In [11]:
#convert moves
moveFrame['to'] = moveFrame['to'].apply(parseMove)
moveFrame['from'] = moveFrame['from'].apply(parseMove)

In [12]:
#convert promotions
promoteFrame['promote'] = promoteFrame['promote'].apply(parsePromote)

In [13]:
#our final frames
print(moveFrame)
print(promoteFrame)

       index                                              board  \
32398  43823  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4670    5763  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
16911  22067  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
17791  23417  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
39673  53906  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
...      ...                                                ...   
27902  37420  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
36897  50341  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
48062  65580  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
18313  24095  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
27207  36496  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                    from  \
32398  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4670   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
16911  [0.0, 0.

In [14]:
#manually fill arrays so we have the right shape, slow and stupid but it works
moveInput = np.empty((51415,512))
toOutput = np.empty((51415,64))
fromOutput = np.empty((51415,64))
promoteInput = np.empty((1148,512))
promoteOutput = np.empty((1148,3))


for i in range(51415):
    array = np.asarray(moveFrame.loc[[i]]["board"])
    for j in range(512):
        moveInput[i][j] = array[0][j]
print("moveInput done")
        
for i in range(51415):
    array = np.asarray(moveFrame.loc[[i]]["to"])
    for j in range(64):
        toOutput[i][j] = array[0][j]
print("toOutput done")

for i in range(51415):
    array = np.asarray(moveFrame.loc[[i]]["from"])
    for j in range(64):
        fromOutput[i][j] = array[0][j]
print("fromOutput done")

for i in range(1148):
    array = np.asarray(promoteFrame.loc[[i]]["board"])
    for j in range(512):
        promoteInput[i][j] = array[0][j]
print("promoteInput done")

for i in range(1148):
    array = np.asarray(promoteFrame.loc[[i]]["promote"])
    for j in range(3):
        promoteOutput[i][j] = array[0][j]
print("promoteOutput done")


moveInput done
toOutput done
fromOutput done
promoteInput done
promoteOutput done


In [30]:
#initialize our models
toModel = keras.Sequential()
fromModel = keras.Sequential()
promoteModel = keras.Sequential()

In [31]:
#define the toModel
toModel.add(keras.layers.Dense(units=100,input_shape=(512,))) #hidden layer 1 with input
toModel.add(keras.layers.Dense(units=100)) #h2
toModel.add(keras.layers.Dense(units=64)) #output layer
toModel.compile(optimizer='sgd',loss='mean_squared_error',metrics=['accuracy'])#we will use sgd and mse to train


#shape:
# 512 -> 100 -> 100 -> 64
# in     h1     h2     out

In [32]:
#train it
toModel.fit(moveInput,toOutput,epochs = 100)

Epoch 1/100
1607/1607 [==============================] - 12s 8ms/step - loss: 0.0387 - accuracy: 0.0183
Epoch 2/100
1607/1607 [==============================] - 11s 7ms/step - loss: 0.0312 - accuracy: 0.0278
Epoch 3/100
1607/1607 [==============================] - 13s 8ms/step - loss: 0.0276 - accuracy: 0.0395
Epoch 4/100
1607/1607 [==============================] - 13s 8ms/step - loss: 0.0252 - accuracy: 0.0534
Epoch 5/100
1607/1607 [==============================] - 13s 8ms/step - loss: 0.0235 - accuracy: 0.0722
Epoch 6/100
1607/1607 [==============================] - 13s 8ms/step - loss: 0.0222 - accuracy: 0.0925
Epoch 7/100
1607/1607 [==============================] - 17s 10ms/step - loss: 0.0212 - accuracy: 0.1133
Epoch 8/100
1607/1607 [==============================] - 16s 10ms/step - loss: 0.0203 - accuracy: 0.1334
Epoch 9/100
1607/1607 [==============================] - 14s 8ms/step - loss: 0.0196 - accuracy: 0.1553
Epoch 10/100
1607/1607 [==============================] - 13s 

1607/1607 [==============================] - 13s 8ms/step - loss: 0.0115 - accuracy: 0.5605
Epoch 80/100
1607/1607 [==============================] - 11s 7ms/step - loss: 0.0115 - accuracy: 0.5613
Epoch 81/100
1607/1607 [==============================] - 12s 7ms/step - loss: 0.0114 - accuracy: 0.5619
Epoch 82/100
1607/1607 [==============================] - 15s 9ms/step - loss: 0.0114 - accuracy: 0.5625
Epoch 83/100
1607/1607 [==============================] - 16s 10ms/step - loss: 0.0114 - accuracy: 0.5631
Epoch 84/100
1607/1607 [==============================] - 16s 10ms/step - loss: 0.0114 - accuracy: 0.5644
Epoch 85/100
1607/1607 [==============================] - 17s 10ms/step - loss: 0.0113 - accuracy: 0.5650
Epoch 86/100
1607/1607 [==============================] - 17s 10ms/step - loss: 0.0113 - accuracy: 0.5654
Epoch 87/100
1607/1607 [==============================] - 14s 9ms/step - loss: 0.0113 - accuracy: 0.5660
Epoch 88/100
1607/1607 [==============================] - 18s 11

In [34]:
#define and train the from model
fromModel.add(keras.layers.Dense(units=100,input_shape=(512,))) #hidden layer 1 with input
fromModel.add(keras.layers.Dense(units=100)) #h2
fromModel.add(keras.layers.Dense(units=64)) #output layer
fromModel.compile(optimizer='sgd',loss='mean_squared_error',metrics=['accuracy'])#we will use sgd and mse to train
fromModel.fit(moveInput,fromOutput,epochs = 100)

Epoch 1/100
1607/1607 [==============================] - 13s 8ms/step - loss: 0.0405 - accuracy: 0.0199
Epoch 2/100
1607/1607 [==============================] - 13s 8ms/step - loss: 0.0322 - accuracy: 0.0268
Epoch 3/100
1607/1607 [==============================] - 13s 8ms/step - loss: 0.0284 - accuracy: 0.0350
Epoch 4/100
1607/1607 [==============================] - 13s 8ms/step - loss: 0.0260 - accuracy: 0.0444
Epoch 5/100
1607/1607 [==============================] - 14s 9ms/step - loss: 0.0243 - accuracy: 0.0542
Epoch 6/100
1607/1607 [==============================] - 14s 8ms/step - loss: 0.0230 - accuracy: 0.0646
Epoch 7/100
1607/1607 [==============================] - 13s 8ms/step - loss: 0.0220 - accuracy: 0.0752
Epoch 8/100
1607/1607 [==============================] - 13s 8ms/step - loss: 0.0211 - accuracy: 0.0865
Epoch 9/100
1607/1607 [==============================] - 13s 8ms/step - loss: 0.0205 - accuracy: 0.0971
Epoch 10/100
1607/1607 [==============================] - 13s 8m

In [35]:
#define and train the promote model
promoteModel.add(keras.layers.Dense(units=100,input_shape=(512,))) #hidden layer 1 with input
promoteModel.add(keras.layers.Dense(units=50)) #h2
promoteModel.add(keras.layers.Dense(units=3)) #output layer
promoteModel.compile(optimizer='sgd',loss='mean_squared_error',metrics=['accuracy'])#we will use sgd and mse to train
promoteModel.fit(promoteInput,promoteOutput,epochs = 100)

Epoch 1/100
36/36 [==============================] - 0s 9ms/step - loss: 0.2332 - accuracy: 0.6254
Epoch 2/100
36/36 [==============================] - 0s 8ms/step - loss: 0.1607 - accuracy: 0.7770
Epoch 3/100
36/36 [==============================] - 0s 8ms/step - loss: 0.1394 - accuracy: 0.7875
Epoch 4/100
36/36 [==============================] - 0s 10ms/step - loss: 0.1240 - accuracy: 0.8014
Epoch 5/100
36/36 [==============================] - 0s 11ms/step - loss: 0.1119 - accuracy: 0.8136
Epoch 6/100
36/36 [==============================] - 0s 9ms/step - loss: 0.1020 - accuracy: 0.8197
Epoch 7/100
36/36 [==============================] - 0s 10ms/step - loss: 0.0935 - accuracy: 0.8354
Epoch 8/100
36/36 [==============================] - 0s 9ms/step - loss: 0.0863 - accuracy: 0.8510
Epoch 9/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0801 - accuracy: 0.8728
Epoch 10/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0747 - accuracy: 0.8850
Epoch 

In [36]:
#save the models
toModel.save("./src/resources/toModel.h5")
fromModel.save("./src/resources/fromModel.h5")
promoteModel.save("./src/resources/promoteModel.h5")